# *SNMP gets a bad wrap.**

We can all agree that "SNMP is the best! I love it!" said no one ever. But SNMP is still the largest set of data available to us that can give us deep insight into the inner workings of our network infrastructure. 

It's not pretty, but it's a virtual treasure trove for those who know how to read the secrets. 

Thankfully, python has the pysnmp library which gives us a way to interact with SNMP in a fairly painless way. Let's be honest though, SNMP is always going to be a little bit painful, right?

To start with this quick tutorial, we're going to first import the high-level API module from the pysnmp library as well as import the *ipaddress* module which is native to python. 

In [1]:
from pysnmp.hlapi import
import ipaddress

Now that we've set the foundation, let's start by creating a new network which we're going to use to iterate over as part of a basic SNMP discovery. 

To do this, we'll create a new object called *my_network* and we'll fill it up with the 10.10.0.0/24 network range. 

We're going to use this range as the IP addresses which we're going to scan using SNMP to create ourselves a list of the devices in this particular subnet that have SNMP turned on.

In [99]:
my_network = ipaddress.ip_network(u'10.101.0.0/24')
my_host = '10.20.10.10' #This was just for testing purposes. Safe to ignore it.

### Creating the *get_sysDescr* function

One of the great things in python is how easy it can be to create a new function that you can quickly accesss to perform a task. 

In our case, we want to discover and create a quick inventory of what devices have SNMP turned on in a specific subnet, which I'm liberally interpreting as grab the contents of the sysDesc mib object which can be found at  **.1.3.6.1.2.1.1.1**

Thankfully, we can use the high-level API from the pysnmp library to make this a bit easier on ourselves. 

I'm not going to go to deep into this code, but you can always check out the pysmp docs for more information on exactly what's going on here. 


In [87]:
def get_sysDescr(ip_address):
    errorIndication, errorStatus, errorIndex, varBinds = next(
    getCmd(SnmpEngine(),
           CommunityData('public', mpModel=0),
           UdpTransportTarget((str(ip_address), 161)),
           ContextData(),
           ObjectType(ObjectIdentity('SNMPv2-MIB', 'sysDescr', 0)))
)

    if errorIndication:
        print(errorIndication)
    elif errorStatus:
        print('%s at %s' % (errorStatus.prettyPrint(),
                        errorIndex and varBinds[int(errorIndex) - 1][0] or '?'))
    else:
        for varBind in varBinds:
            x = (' = '.join([x.prettyPrint() for x in varBind]))
        return x

### Testing our new function

Using the *my_host* varible above to make sure that we're getting the information we expect. 

If this works, we should see in the output that this is an HPE 5900AF switch. 

In [100]:
get_sysDescr(my_host)

u'SNMPv2-MIB::sysDescr.0 = HPE Comware Platform Software, Software Version 7.1.045, Release 2432P02\r\nHPE 5900AF-48XGT-4QSFP+ Switch\r\nCopyright (c) 2010-2017 Hewlett Packard Enterprise Development LP'

Now that it works. We can use it to do a quick sweep through the whole subnet and create a list of all the devices that respond to SNMP with the same string. 

Note - There's DEFINITELY the opportunity for some optimization here. The code as written operates in serial and there are timeouts involved as well for addresses which don't respond. 

In [94]:
devices = []
for i in my_network:
    dev = get_sysDescr(i)
    if dev != None:
        if len(dev) > 0:
            dev = dev.split('=')
            devices.append(dev[1])

No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response

No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response

Now that we've collected the sysDescr from these devices, we can now use them in any way 
that python allows us to including
- print them
- copy them into a spreadsheet
- use them as input to another function

This is the beauty of applying code to our network operations model is that it gives us power over our environment in ways that off-the-shelf tools just can't deliver. 

In [96]:
for i in devices:
    print(i)

 Cisco IOS Software, 2800 Software (C2800NM-ADVENTERPRISEK9-M), Version 15.1(4)M, RELEASE SOFTWARE (fc1)
Technical Support: http://www.cisco.com/techsupport
Copyright (c) 1986-2011 by Cisco Systems, Inc.
Compiled Thu 24-Mar-11 14:26 by prod_rel_team
 TippingPoint IPS
 VMware ESXi 5.5.0 build-2403361 VMware, Inc. x86_64
 VMware ESXi 5.5.0 build-2403361 VMware, Inc. x86_64
 Integrated Lights-Out 4 2.50 Sep 23 2016
 Integrated Lights-Out 4 2.40 Dec 02 2015
 Sentry Switched CDU
 Sentry Switched CDU
 HP ETHERNET MULTI-ENVIRONMENT
 Linux HPLAB 2.6.32.12 #15217 Fri Nov 10 20:07:44 CST 2017 armv5tel
 Linux DiskStation 2.6.32.12 #15217 Fri Nov 10 20:07:44 CST 2017 armv5tel
 UAP-AC-LR 3.9.19.8123
 3Com-NJ2000 - 3CNJ2000 v2.00.04
 HP Comware Platform Software, Software Version 5.20.99 Release 2221P20
HP A5500-24G-PoE+ EI Switch with 2 Interface Slots
Copyright (c) 2010-2015 Hewlett Packard Enterprise Development LP
 3Com Switch 4210 PWR 9-Port Software Version 3Com OS V3.01.13s168
 HP Comware Pla

# With a bow on top

SNMP is not pretty. SNMP is not easy. It's not "simple" and the "M" in SNMP does not stand for magic. We would all love it to go away. 

All that being said, it _is_ useful and in a lot of cases, it may be the only way for us to get to our network data without having to resort to screen scraping and regular expressions.

